In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool

In [2]:
load_dotenv()

True

# Creating subagents

In [3]:
@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""

    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [9]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")

In [10]:
# Create subagents

subagent_1 = create_agent(
    model=model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model = model,
    tools = [square]
)

# Calling subagents

In [11]:
@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})

    return response["messages"][-1].content


@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 1 in order to calculate the square of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})

    return response["messages"][-1].content

In [12]:
# Create main agent

main_agent = create_agent(
    model=model,
    tools = [call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of number."
)

# Test

In [13]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages":[HumanMessage(content=question)]})

In [14]:
from pprint import pprint
pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='d417aa68-1289-42b8-bcd5-1d7cd30538a4'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'call_subagent_1', 'arguments': '{"x": 456}'}, '__gemini_function_call_thought_signatures__': {'ba919e02-895e-4740-9397-293dc9b81cac': 'CvUBAXLI2nzdF2aBiV98m4QlmTFBIrnWakHxnW3SsicN5SrvbOY2f9OCpwUQ9+yQFWe983KgXH3CdVMM9dCX81Mpm8D20QlfQ0oH5wz0b69dgnltaTfIxR05BeSZ/JVsa+y04Jm28w/LeeyuFrs51Am1Pgc0swNXFFEd4UiiEMLkpVq/W6FeDeL43fQh/VOItbiMdJGVLr5Kdle0oZIE1DRgV/t+TYwkROk8Gsamj/2dBaSUbbNtsjGUCylE2PO8C7NKTnYYRs3+H8fn5qXB1J4OO6beXzIqVvRCqTHb/0GYnYOct3/vROJaDpYf2ypu0YsfrqlvzJk='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bcf85-c959-71c3-9c9c-190a3cc1139d-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'ba919e02-895e-4740-9397-293dc9b81